In [ ]:
# Code for artery tracking 

In [ ]:
#set CUDA_VISIBLE_DEVICES=1

In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import json
import cv2
import os
import matplotlib.pyplot as plt
import copy
import numpy as np
import pickle
import glob
import datetime
import pickle


In [ ]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');

In [ ]:
taskdir = '//Desktop4/Dtensorflow\\LiChen\\AICafe\\CNNTracker'
taskname = 'CNNTracker1-1'
if not os.path.exists(taskdir+'/'+taskname):
    os.mkdir(taskdir+'/'+taskname)

In [ ]:
import sys
sys.path.append(r'U:\LiChen\AICafe\CNNTracker')
from models.centerline_net import CenterlineNet
from centerline_train_tools.data_provider_argu import DataGenerater
from centerline_train_tools.centerline_trainner import Trainer
import torch

In [ ]:
#import iCafe Python
import numpy as np
import sys
#sys.path.append(r'\\DESKTOP2\Ftensorflow\LiChen\iCafe')
sys.path.insert(0,r'\\DESKTOP4\Dtensorflow\LiChen\iCafePython')
from iCafePython import iCafe
from iCafePython import SnakeList,Snake,SWCNode,Point3D

# Dataset

In [ ]:
dbname = 'BRAVEAI'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'+dbname
with open(r"\\DESKTOP2\GiCafe\result\BRAVEAI\db.list",'rb') as fp:
    dblist = pickle.load(fp)
train_list = dblist['train']
val_list = dblist['val']
test_list = dblist['test']
pilist = [pi.split('/')[1] for pi in dblist['test']]
len(pilist),pilist

In [ ]:
dbname = 'RotterdanCoronary'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'+dbname
pilist = ['0_dataset03_U']
icafem = iCafe(icafe_dir+'/'+pilist[0])

In [ ]:
dbname = 'UNC'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'+dbname
with open(icafe_dir+'/db.list','rb') as fp:
    dblist = pickle.load(fp)
pilist = [pi.split('/')[1] for pi in dblist['test']]
len(pilist)

In [ ]:
dbname = 'HarborViewT1Pre'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'+dbname

pilist = ['0_ID%d_U'%i for i in [2,9,10,11,12]]
len(pilist)

In [ ]:
# MERGE 
dbname = 'CAREIIMERGEGT'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'+dbname
with open(icafe_dir+'/db.list','rb') as fp:
    dblist = pickle.load(fp)
pilist = [pi.split('/')[1] for pi in dblist['test']]
len(pilist)

In [ ]:
# IPH supp
dbname = 'IPH-Sup-TOF-FullCoverage'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'+dbname

dblist_name = icafe_dir+'/db.list'

with open(dblist_name,'rb') as fp:
    dblist = pickle.load(fp)
        
pilist = [pi.split('/')[1] for pi in dblist['test']]
len(pilist)

## Prepare dataset

In [ ]:
pilist = [pi.split('/')[1] for pi in dblist['train']]+[pi.split('/')[1] for pi in dblist['val']]+[pi.split('/')[1] for pi in dblist['test']]

In [ ]:
for pi in pilist[:]:
    tif_name = icafe_dir + '/' + pi + '/TH_' + pi + '.npy'
    if 1:
        icafem = iCafe(icafe_dir +'/'+pi)
        icafem.loadImg('o')
        

        tif_norm_img = copy.copy(icafem.tifimg)
        tif_norm_img = tif_norm_img/1000
        print(pi,np.max(tif_norm_img))
        np.save(icafem.getPath('.npy'), tif_norm_img.astype(np.float16))

# Distance transfrom

In [ ]:
import shutil
for pi in pilist[:]:
    
    icafem = iCafe(icafe_dir +'/'+pi)
    if icafem.existPath('d.npy'):
        continue
    print(pi)
    #ves
    #snakelist = icafem.vessnakelist
    #raw_ves
    snakelist = icafem.snakelist
    icafem.paintDistTransform(snakelist)


# Train

In [ ]:
def get_dataset(train_patch_list,val_patch_list):
    '''
    :return: train set,val set
    '''
    train_pre_fix_path = r"D:\LiChen\BRAVEpatch"
    train_flag = 'train'
    train_transforms = None
    target_transform = None
    train_dataset = DataGenerater(train_patch_list, train_pre_fix_path, 500, train_transforms, train_flag, target_transform)

    val_pre_fix_path = r"D:\LiChen\BRAVEpatch"
    val_flag = 'val'
    test_valid_transforms = None
    target_transform = None
    val_dataset = DataGenerater(val_patch_list, val_pre_fix_path, 500, test_valid_transforms, val_flag, target_transform)

    return train_dataset, val_dataset


def cross_entropy(a, y):
    epsilon = 1e-9
    return torch.mean(torch.sum(-y * torch.log10(a + epsilon) - (1 - y) * torch.log10(1 - a + epsilon), dim=1))


In [ ]:
list_dir = r'D:\LiChen\BRAVEpatch\icafe_patch\offset\point_500_gp_1'
train_patch_list = []
val_patch_list = []
for pi in train_list:
    if os.path.exists(list_dir+'/d'+pi.split('/')[1]+'_patch_info_500.csv'):
        train_patch_list.append(list_dir+'/d'+pi.split('/')[1]+'_patch_info_500.csv')
for pi in val_list:
    if os.path.exists(list_dir+'/d'+pi.split('/')[1]+'_patch_info_500.csv'):
        val_patch_list.append(list_dir+'/d'+pi.split('/')[1]+'_patch_info_500.csv')
train_patch_list,val_patch_list

In [ ]:
# Here we use 8 fold cross validation, save_num means to use dataset0x as the validation set
train_dataset, val_dataset = get_dataset(train_patch_list[:],val_patch_list[:])

curr_model_name = "centerline_net"
max_points = 500
model = CenterlineNet(n_classes = max_points)

batch_size = 64
num_workers = 16

criterion = cross_entropy
inital_lr = 0.001

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=inital_lr,weight_decay=0.001)

trainer = Trainer(batch_size,
                  num_workers,
                  train_dataset,
                  val_dataset,
                  model,
                  curr_model_name,
                  optimizer,
                  criterion,
                  max_points,
                  start_epoch=38,
                  max_epoch=100,
                  initial_lr=inital_lr)
trainer.run_train()

In [ ]:
trainer = Trainer(batch_size,
                  num_workers,
                  train_dataset,
                  val_dataset,
                  model,
                  curr_model_name,
                  optimizer,
                  criterion,
                  max_points,
                  start_epoch=0,
                  max_epoch=100,
                  initial_lr=inital_lr)

trainer.run_train()

In [ ]:
import SimpleITK as sitk
data_path = r"D:\LiChen\BRAVEpatch\icafe_patch\offset\point_500_gp_1\d0_7001_U\d_0_7001_U_v_0_patch_1_0.nii.gz"
img = sitk.GetArrayFromImage(sitk.ReadImage(data_path, sitk.sitkFloat32))

In [ ]:
plt.figure(figsize=(10,5))
plt.imshow(img.reshape(19,-1))

# CNN Tracker

In [ ]:
#CNN tracker for ICA TOF MRA
swc_name = 'cnn_snake'

import sys
sys.path.append(r'U:\LiChen\AICafe\CNNTracker')
from models.centerline_net import CenterlineNet

max_points = 500
prob_thr = 0.85

infer_model = CenterlineNet(n_classes=max_points)
checkpoint_path_infer = r"D:\tensorflow\LiChen\AICafe\CNNTracker\CNNTracker1-1\classification_checkpoints\centerline_net_model_Epoch_29.pkl"

checkpoint = torch.load(checkpoint_path_infer)
net_dict = checkpoint['net_dict']
infer_model.load_state_dict(net_dict)
infer_model.to(device)
infer_model.eval()

In [ ]:
#CNN tracker for Coronary CTA
swc_name = 'cnn_snake'
max_points = 500
prob_thr = 0.85

infer_model = CenterlineNet(n_classes=max_points)

checkpoint_path_infer = r"D:\tensorflow\LiChen\AICafe\CNNTracker\CNNTracker2-1\classification_checkpoints\centerline_net_model_Epoch_81.pkl"

checkpoint = torch.load(checkpoint_path_infer)
net_dict = checkpoint['net_dict']
infer_model.load_state_dict(net_dict)
infer_model.to(device)
infer_model.eval()

In [ ]:
#CNN tracker for LATTE
swc_name = 'cnn_snake'
max_points = 500
prob_thr = 0.85

infer_model = CenterlineNet(n_classes=max_points)
checkpoint_path_infer = r"D:\tensorflow\LiChen\AICafe\CNNTracker\CNNTracker4-1\classification_checkpoints\centerline_net_model_Epoch_99.pkl"



checkpoint = torch.load(checkpoint_path_infer)
net_dict = checkpoint['net_dict']
infer_model.load_state_dict(net_dict)
infer_model.to(device)
infer_model.eval()

In [ ]:
#DCAT for ICA TOF MRA
swc_name = 'dcat_snake'

from models.dcat_net import DCATNet
checkpoint_path_infer = r"D:\tensorflow\LiChen\AICafe\CNNTracker\CNNTracker3-1\classification_checkpoints\DCAT_net_model_Epoch_7.pkl"
dcat_model = DCATNet(n_classes=max_points)
checkpoint = torch.load(checkpoint_path_infer)
net_dict = checkpoint['net_dict']
dcat_model.load_state_dict(net_dict)

max_points = 500
prob_thr = 0.85

infer_model = CenterlineNet(n_classes=max_points)

model_dict = infer_model.state_dict()
pretrained_dict = dcat_model.state_dict()
# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict) 
# 3. load the new state dict
infer_model.load_state_dict(pretrained_dict)

infer_model.to(device)
infer_model.eval()

In [ ]:
swc_name

In [ ]:
dbname = 'BRAVEAI'
import SimpleITK as sitk
from infer_tools_tree.build_vessel_tree import build_vessel_tree, TreeNode, dfs_search_tree

icafe_dir = r'\\DESKTOP2\GiCafe\result/BRAVEAI/'
for pi in pilist[:]:
    print('='*10,'Start processing',pilist.index(pi),'/',len(pilist),pi,'='*10)

    icafem = iCafe(icafe_dir+'/'+pi)
    
    #load image

    file_name = icafem.getPath('o')
    re_spacing_img = sitk.GetArrayFromImage(sitk.ReadImage(file_name))
    
    # generate/load seeds
    from skimage.morphology import skeletonize,skeletonize_3d
    from iCafePython.utils.img_utils import connectedCentroid
    if icafem.existPath('con_seed.txt'):
        con_seeds = np.loadtxt(icafem.getPath('con_seed.txt'))
    else:
        simg = icafem.loadImg('s.whole')>0.5
        skeleton = skeletonize_3d(simg)
        con_seeds = connectedCentroid(skeleton)
        print('len',len(con_seeds))
        np.savetxt(icafem.getPath('con_seed.txt'),con_seeds)

    seeds = np.vstack(([i.intlst() for i in icafem.seeds], con_seeds))
    print(len(seeds))
    
    #tracing
    ostias = seeds[:1]
    root = TreeNode(ostias, start_point_index=None)
    all_traces = build_vessel_tree(infer_model, re_spacing_img, seeds[::], root=root)
    #convert to snakelist
    cnn_snakelist = SnakeList()
    for snakei in all_traces:
        cnn_snake = Snake()
        for x,y,z,r in snakei:
            cnn_snake.addSWC(Point3D(x,y,z),r)
        cnn_snake = cnn_snake.trimRange(icafem.box)
        cnn_snakelist.addSnake(cnn_snake.resampleSnake(1))

    cnn_snakelist = cnn_snakelist.trimDuplicateSnake(icafem.shape)
    icafem.writeSWC(swc_name,cnn_snakelist)
    print(cnn_snakelist)

In [ ]:
#search vessel to ostias based on relation in root
if 0:
    single_tree = dfs_search_tree(root)
    vessel_tree_postprocess = []
    for vessel_list in single_tree:
        vessel_list.pop(0)
        res = np.array([]).reshape(0, 4)
        while vessel_list:
            first_node = vessel_list[0]
            first_res = np.hstack((first_node.value,first_node.rad))
            vessel_list.pop(0)
            if vessel_list:
                second_node = vessel_list[0]
                first_res = first_res[:second_node.start_point_index]
                res = np.vstack((res, first_res))
            else:
                res = np.vstack((res, first_res))
                vessel_tree_postprocess.append(res)
    cnn_snakelist = SnakeList()
    for snakei in vessel_tree_postprocess:
        cnn_snake = Snake()
        for x,y,z,r in snakei:
            cnn_snake.addSWC(Point3D(x,y,z),r)
        cnn_snakelist.addSnake(cnn_snake.resampleSnake(1))
    cnn_snakelist.trimDuplicateSnake(icafem.shape)
    icafem.writeSWC('cnn_ves_snake',cnn_snakelist)

# Extend snake

In [ ]:
icafem = iCafe(icafe_dir+'/'+pilist[1])
icafem

In [ ]:
#load image
import SimpleITK as sitk
file_name = icafem.getPath('o')
re_spacing_img = sitk.GetArrayFromImage(sitk.ReadImage(file_name))
icafem,re_spacing_img.shape

In [ ]:
from iCafePython.connect.ext import extSnake
seg_ves_snakelist = SnakeList()
csnake = Snake()
csnake.addSWC(Point3D(424, 40, 406),1)
csnake.addSWC(Point3D(423, 41, 410),1)
seg_ves_snakelist.addSnake(csnake)
seg_ves_ext_snakelist = extSnake(seg_ves_snakelist,infer_model,re_spacing_img,DEBUG=0)

In [ ]:
from iCafePython.connect.ext import createLabelMap
traced_img = createLabelMap(seg_ves_snakelist, re_spacing_img.shape)
snakei = 195
seg_ves_snake = seg_ves_snakelist[snakei]
chead = seg_ves_snake[0].pos.lst()
move_dir_head = (seg_ves_snake[0].pos - seg_ves_snake[1].pos).norm()

ctail = seg_ves_snake[-1].pos.lst()
move_dir_tail = (seg_ves_snake[-1].pos - seg_ves_snake[-2].pos).norm()

# add seg ahead of head of snake
# set starting point as -1 to allow stretching
paint_ball(traced_img, seg_ves_snake[0].pos, seg_ves_snake[0].rad, -1)
add_seg = extend_seg_end(infer_model, re_spacing_img, traced_img, chead, move_dir_head)

In [ ]:
from iCafePython.connect.ext import extSnake
seg_ves_snakelist = icafem.readSnake('seg_ves')
seg_ves_ext_snakelist = extSnake(seg_ves_snakelist,infer_model,re_spacing_img,DEBUG=0)
icafem.writeSWC('seg_ves_ext',seg_ves_ext_snakelist)

## steps

In [ ]:
#coronary original model
checkpoint_path_infer = r"U:\LiChen\AICafe\CNNTracker\checkpoint\classification_checkpoints\centerline_net_model_s1.pkl"
max_points = 500
prob_thr = 0.85
infer_model = CenterlineNet(n_classes=max_points)
checkpoint = torch.load(checkpoint_path_infer)
net_dict = checkpoint['net_dict']
infer_model.load_state_dict(net_dict)
infer_model.to(device)
infer_model.eval()

In [ ]:
sys.path.append(r'U:\LiChen\AICafe\CNNTracker\infer_tools_tree')
from infer_tools_tree.build_vessel_tree import build_vessel_tree, TreeNode, dfs_search_tree, search_first_node

In [ ]:


#load image
import SimpleITK as sitk
file_name = icafem.getPath('o')
re_spacing_img = sitk.GetArrayFromImage(sitk.ReadImage(file_name))
re_spacing_img.shape



In [ ]:
def paint_ball(img_fill, ct, rad, target=0):
    ct_int = ct.intlst()
    rad_int = int(round(rad))
    posz = min(img_fill.shape[0]-1,ct_int[2])
    posx = min(img_fill.shape[1]-1,ct_int[0])
    posy = min(img_fill.shape[2]-1,ct_int[1])
    img_fill[posz,posx,posy] = 0
    for ofx in range(-rad_int, rad_int):
        for ofy in range(-rad_int, rad_int):
            for ofz in range(-rad_int, rad_int):
                cpos = Point3D([ct_int[0] + ofx, ct_int[1] + ofy, ct_int[2] + ofz])
                cpos.boundList([img_fill.shape[1],img_fill.shape[2],img_fill.shape[0]])
                cdist = cpos.dist(ct)
                if cdist > rad:
                    continue
                posz = min(img_fill.shape[0]-1,cpos.intlst()[2])
                posx = min(img_fill.shape[1]-1,cpos.intlst()[0])
                posy = min(img_fill.shape[2]-1,cpos.intlst()[1])
                img_fill[posz,posx,posy] = target
                    
def createLabelMap(seg_ves_snakelist,shape):
    label_img = np.ones(shape, dtype=np.int16)*(-1)
    for snakeid in range(seg_ves_snakelist.NSnakes):
        if snakeid%5==0:
            print('\rpainting snake',snakeid,end='')
        for pti in range(seg_ves_snakelist._snakelist[snakeid].NP):
            pos = seg_ves_snakelist._snakelist[snakeid][pti].pos
            rad = seg_ves_snakelist._snakelist[snakeid][pti].rad
            # paint within radius at pos position
            paint_ball(label_img, pos, rad, 0)
    return label_img

from infer_tools_tree.build_vessel_tree import search_first_node, search_one_direction
def extend_seg_end(infer_model, re_spacing_img, traced_img, chead, cdir_mv):
    #chead: current segment ending position
    #cdir_mv: move direction based on first two pts on the original segment
    prob_records=[0,0,0]
    res = search_first_node(infer_model, re_spacing_img, start=chead, prob_records=prob_records)
    add_seg = Snake()
    if res is None:
        print('not valid patch')
    else:
        direction, prob_records, curr_r = res
        if prob_records[-1]>prob_thr:
            print('head prob',prob_records[-1],'higher than termination prob_thr',prob_thr)
        else:
            forward_matchness = Point3D(direction['forward_vector']).norm()*cdir_mv
            backward_matchness = Point3D(direction['backward_vector']).norm()*cdir_mv
            if forward_matchness>backward_matchness:
                sel_start_pos = direction['forward']
                sel_direction = direction['forward_vector']
            else:
                sel_start_pos = direction['backward']
                sel_direction = direction['backward_vector']

            point_list = []
            r_list = []
            
            find_node = search_one_direction(infer_model, traced_img, re_spacing_img, start=sel_start_pos,
                                                 move_direction=sel_direction,
                                                 prob_records=prob_records,
                                                 r_list=r_list, point_list=point_list,step_ratio=0.5)
            for pti in range(len(point_list)):
                add_seg.addSWC(Point3D(point_list[pti]),r_list[pti])
    return add_seg        

In [ ]:
plt.imshow(np.max(traced_img,axis=0))
plt.savefig('1.png')
plt.show()

In [ ]:

seg_ves_snakelist = icafem.readSnake('seg_ves')
#traced_img = seg_ves_snakelist.labelMap(icafem.shape)
traced_img = createLabelMap(seg_ves_snakelist,re_spacing_img.shape)

for snakei in range(seg_ves_snakelist.NSnakes):
    seg_ves_snake = seg_ves_snakelist[snakei]
    chead = seg_ves_snake[0].pos.lst()
    move_dir_head = (seg_ves_snake[0].pos-seg_ves_snake[1].pos).norm()

    ctail = seg_ves_snake[-1].pos.lst()
    move_dir_tail = (seg_ves_snake[-1].pos-seg_ves_snake[-2].pos).norm()
    
    #add seg ahead of head of snake
    #set starting point as -1 to allow stretching
    paint_ball(traced_img, seg_ves_snake[0].pos, seg_ves_snake[0].rad, -1)
    add_seg = extend_seg_end(infer_model, re_spacing_img, traced_img, chead, move_dir_head)
    #restore
    paint_ball(traced_img, seg_ves_snake[0].pos, seg_ves_snake[0].rad, 0)
    if add_seg.NP>0:
        print(snakei,'head prepend',add_seg)
        seg_ves_snake.mergeSnake(add_seg,reverse=False,append=False)
    else:
        print(snakei,'head no extend')
    #add seg after tail of snake
    paint_ball(traced_img, seg_ves_snake[-1].pos, seg_ves_snake[-1].rad, -1)
    add_seg = extend_seg_end(infer_model, re_spacing_img, traced_img, ctail, move_dir_tail)
    paint_ball(traced_img, seg_ves_snake[-1].pos, seg_ves_snake[-1].rad, 0)
    if add_seg.NP>0:
        print(snakei,'tail add',add_seg)
        seg_ves_snake.mergeSnake(add_seg,reverse=False,append=True)
    else:
        print(snakei,'tail no extend')
        
icafem.writeSWC('seg_ves_ext',seg_ves_snakelist)

In [ ]:
#debug
seg_ves_snakelist = icafem.readSnake('seg_ves')
#traced_img = seg_ves_snakelist.labelMap(icafem.shape)
traced_img = createLabelMap(seg_ves_snakelist,re_spacing_img.shape)


seg_ves_snakelist = icafem.readSnake('seg_ves')
snakei = 175
seg_ves_snake = seg_ves_snakelist[snakei]

if 1:
    chead = seg_ves_snake[0].pos.lst()

    move_dir_head = (seg_ves_snake[1].pos-seg_ves_snake[0].pos).norm()

    paint_ball(traced_img, seg_ves_snake[0].pos, seg_ves_snake[0].rad, -1)

    prob_records=[0,0,0]
    res = search_first_node(infer_model, re_spacing_img, start=chead, prob_records=prob_records)
    direction, prob_records, curr_r = res
    print('prob',prob_records[-1])

    cdir_mv = move_dir_head
    
else:
    
    ctail = seg_ves_snake[-1].pos.lst()
    move_dir_tail = (seg_ves_snake[-1].pos-seg_ves_snake[-2].pos).norm()

    paint_ball(traced_img, seg_ves_snake[-1].pos, seg_ves_snake[-1].rad, -1)

    prob_records=[0,0,0]
    res = search_first_node(infer_model, re_spacing_img, start=ctail, prob_records=prob_records)
    direction, prob_records, curr_r = res
    print('prob',prob_records[-1])
    cdir_mv = move_dir_tail

forward_matchness = Point3D(direction['forward_vector']).norm()*cdir_mv
backward_matchness = Point3D(direction['backward_vector']).norm()*cdir_mv

print(forward_matchness,backward_matchness)
if forward_matchness>backward_matchness:
    sel_start_pos = direction['forward']
    sel_direction = direction['forward_vector']
else:
    sel_start_pos = direction['backward']
    sel_direction = direction['backward_vector']
print(sel_start_pos,'sel pos')

point_list = []
r_list = []

find_node = search_one_direction(infer_model, traced_img, re_spacing_img, start=sel_start_pos,
                                     move_direction=sel_direction,
                                     prob_records=prob_records,
                                     r_list=r_list, point_list=point_list)

len(point_list)

# AICafe

In [ ]:
dbname = 'BRAVEAI'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'+dbname
seg_model_name = 'LumenSeg2-3'

In [ ]:

dbname = 'RotterdanCoronary'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'+dbname
pilist = ['0_dataset05_U']
seg_model_name = 'CoronarySeg1-8-5'


In [ ]:
dbname = 'UNC'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'+dbname
seg_model_name = 'LumenSeg5-1'

In [ ]:
dbname = 'CAREIIMERGEGT'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'+dbname
seg_model_name = 'LumenSeg6-1'

In [ ]:
dbname = 'IPH-Sup-TOF-FullCoverage'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'+dbname
seg_model_name = 'LumenSeg7-1'

In [ ]:
# from s.whole.modelname
from iCafePython.connect.ext import extSnake
import SimpleITK as sitk

RETRACE = 1
RETREE = 1

seg_src = 's.whole.'+seg_model_name
DEBUG = 0

#max search range in merge/branch, unit in mm
search_range_thres = 10

for pi in pilist[1:]:
    print('='*10,'Start processing',pilist.index(pi),'/',len(pilist),pi,'='*10)
    if not os.path.exists(icafe_dir+'/'+pi):
        os.mkdir(icafe_dir+'/'+pi)
        
    icafem = iCafe(icafe_dir+'/'+pi)
    
    icafem.loadImg(seg_src)
    icafem.saveImg('s.whole',icafem.I[seg_src],np.float16)
    icafem.loadImg('s.whole')

    #export v.tif for 3d visualization
    if 'v' not in icafem.listAvailImgs():
        vimg = copy.copy(icafem.I['s.whole'])
        vimg[vimg<0] = 0
        vimg = (vimg*255).astype(np.uint16)
        icafem.saveImg('v',vimg,np.int16)
        
        
    #combined operations
    if RETRACE or not icafem.existPath('seg_ves_ext.swc'):
        if 's.whole' not in icafem.I:
            icafem.loadImg('s.whole')
        seg_ves_snakelist = icafem.constructSkeleton(icafem.I['s.whole']>0.3)
        
        #load image
        file_name = icafem.getPath('o')
        re_spacing_img = sitk.GetArrayFromImage(sitk.ReadImage(file_name))

        seg_ves_snakelist = icafem.readSnake('seg_ves')
        seg_ves_ext_snakelist = extSnake(seg_ves_snakelist,infer_model,re_spacing_img,DEBUG=DEBUG)
        icafem.writeSWC('seg_ves_ext',seg_ves_ext_snakelist)
    else:
        seg_ves_ext_snakelist = icafem.readSnake('seg_ves_ext')
        print('read from existing seg ves ext')
    if seg_ves_ext_snakelist.NSnakes==0:
        print('no snake found in seg ves, abort',pi)
        continue
    
    if RETREE or not icafem.existPath('seg_ves_ext_tree.swc'):
        if 's.whole' not in icafem.I:
            icafem.loadImg('s.whole')
        tree_snakelist = seg_ves_ext_snakelist.tree(icafem,search_range=search_range_thres/icafem.xml.res,int_src='o',DEBUG=DEBUG)
        icafem.writeSWC('seg_ves_ext_tree', tree_snakelist)
        tree_snakelist = tree_snakelist.tree(icafem,search_range=search_range_thres/3/icafem.xml.res,int_src='s.whole',DEBUG=DEBUG)
        icafem.writeSWC('seg_ves_ext_tree2', tree_snakelist)
        
        tree_main_snakelist = tree_snakelist.mainArtTree(dist_thres=10)
        icafem.writeSWC('seg_ves_ext_tree2_main',tree_main_snakelist)
        

In [ ]:
pi = pilist[0]
icafem = iCafe(icafe_dir+'/'+pi)
_ = icafem.loadImg('s.whole')
tree_snakelist = seg_ves_snakelist.tree(icafem,search_range=search_range_thres/icafem.xml.res,int_src='o',DEBUG=DEBUG)
        

In [ ]:
seg_ves_ext_snakelist = icafem.readSnake('seg_ves_ext')
tree_snakelist = seg_ves_ext_snakelist.tree(icafem,search_range=search_range_thres/icafem.xml.res,int_src='o',DEBUG=DEBUG)
icafem.writeSWC('seg_ves_ext_tree', tree_snakelist)

In [ ]:
tree_snakelist.autoBranch()
tree_snakelist.autoTransform(mode='length')
icafem.writeSWC('seg_ves_ext_tree', tree_snakelist)

In [ ]:
from iCafePython.artlabel.artlabel import ArtLabel
art_label_predictor = ArtLabel()

for pi in pilist[:1]:
    print('='*10,'Start processing',pilist.index(pi),'/',len(pilist),pi,'='*10)
    if not os.path.exists(icafe_dir+'/'+dbname+'/'+pi):
        os.mkdir(icafe_dir+'/'+dbname+'/'+pi)
        
    icafem = iCafe(icafe_dir+'/'+dbname+'/'+pi)
    
    #generate (simplified node!=2) graph for GNN art labeling
    G = icafem.generateGraph(graph_ves,None,graphtype='graphsim', mode='test', trim=1)
    if len(G.nodes())<5:
        print('too few snakes for artlabeling')
        continue
    icafem.writeGraph(G,graphtype='graphsim')

    #predict landmarks
    pred_landmark, ves_end_pts = art_label_predictor.pred(icafem.getPath('graphsim'),icafem.xml.res)
    #complete graph Gcom for finding the pts in the path
    Gcom = icafem.generateGraph(graph_ves, None, graphtype='graphcom')
    ves_snakelist = findSnakeFromPts(Gcom,G,ves_end_pts)
    print('@@@predict',len(pred_landmark),'landmarks',ves_snakelist)
    #save landmark and ves
    icafem.xml.landmark = pred_landmark
    icafem.xml.writexml()
    icafem.writeSWC('ves_pred', ves_snakelist)

## sel branch for Harborview

In [ ]:
icafem = iCafe(icafe_dir+'/'+'0_ID12_U')
seg_ves_snakelist = icafem.readSnake('seg_ves_ext')
long_snake_ids = [i for i in range(seg_ves_snakelist.NSnakes) if seg_ves_snakelist[i].length>75]
long_snake_ids

In [ ]:
tree_main_snakelist = seg_ves_snakelist.mainArtTree(dist_thres=10)
snake_hash = [tree_main_snakelist[i][0].pos.hashPos()+tree_main_snakelist[i][-1].pos.hashPos() for i in range(tree_main_snakelist.NSnakes)]
for sid in long_snake_ids:
    if seg_ves_snakelist[sid][0].pos.hashPos()+seg_ves_snakelist[sid][-1].pos.hashPos() in snake_hash:
        continue
    if seg_ves_snakelist[sid][0].pos.x<75:
        continue
    tree_main_snakelist.addSnake(seg_ves_snakelist[sid])
icafem.writeSWC('tree_main',tree_main_snakelist)

# Eval

In [ ]:
def eval_simple(snakelist):
    snakelist = copy.deepcopy(snakelist)
    _ = snakelist.resampleSnakes(1)
    #ground truth snakelist from icafem.veslist
    all_metic = snakelist.motMetric(icafem.veslist)
    metric_dict = all_metic.metrics(['MOTA','IDF1','MOTP','IDS'])
    #ref_snakelist = icafem.readSnake('ves')
    snakelist.compRefSnakelist(icafem.vessnakelist)
    metric_dict['OV'], metric_dict['OF'], metric_dict['OT'], metric_dict['AI'], metric_dict['UM'], metric_dict['UMS'], metric_dict['ref_UM'], metric_dict['ref_UMS'], metric_dict['mean_diff'] = snakelist.evalCompDist()
    str = ''
    metric_dict_simple = ['MOTA','IDF1','MOTP','IDS','OV']
    for key in metric_dict_simple:
        str += key+'\t'
    str += '\n'
    for key in metric_dict_simple:
        if type(metric_dict[key]) == int:
            str += '%d\t'%metric_dict[key]
        else:
            str += '%.3f\t'%metric_dict[key]
    print(str)
    return metric_dict

In [ ]:
pilist

In [ ]:
# calculate metric and save in each pi folder
REFEAT = 0
for pi in pilist[:1]:
    print('='*10,'Start processing',pilist.index(pi),'/',len(pilist),pi,'='*10)
        
    icafem = iCafe(icafe_dir+'/'+pi)
    
    if REFEAT or not icafem.existPath('metric.pickle'):
        print('init metric')
        all_metric_dict = {}
    else:
        print('load metric')
        with open(icafem.getPath('metric.pickle'),'rb') as fp:
            all_metric_dict = pickle.load(fp)
    
    for vesname in ['seg_ves_ext_tree2_main']:
    #for vesname in ['seg_raw','seg_ves_ext_main','seg_ves_ext_tree2']:
    #comparison methods
    #for vesname in ['frangi_ves','seg_unet','seg_raw','raw_sep','cnn_snake','dcat_snake','seg_ves_ext_tree2_main']:
        #if vesname in all_metric_dict:
        #    continue
        print('-'*10,vesname,'-'*10)
        pred_snakelist = icafem.readSnake(vesname)
        if pred_snakelist.NSnakes==0:
            print('no snake',pi,vesname)
            continue
        all_metric_dict[vesname] = eval_simple(pred_snakelist.resampleSnakes(1))
        
    with open(icafem.getPath('metric.pickle'),'wb') as fp:
        pickle.dump(all_metric_dict,fp)
    

In [ ]:
with open(icafem.getPath('metric.old.pickle'),'rb') as fp:
    all_metric_dict_old = pickle.load(fp)
all_metric_dict_old['seg_ves_ext_tree2_main']

In [ ]:
all_metric_dict['seg_ves_ext_tree2_main']

In [ ]:
#check feat
pi = pilist[0]
icafem = iCafe(icafe_dir+'/'+pi)
with open(icafem.getPath('metric.pickle'),'rb') as fp:
    all_metric_dict = pickle.load(fp)
all_metric_dict

In [ ]:
#collect feats from pickle
eval_vesname = {'frangi_ves':'Frangi','seg_unet':'U-Net','seg_raw':'DDT',
                'raw_sep':'iCafe','cnn_snake':'CNN Tracker','dcat_snake':'DCAT','seg_ves_ext_tree2_main':'DOST (ours)',
               'seg_ves':'DOST (initial curve)','seg_ves_ext_main':'DOST (deep snake)','seg_ves_ext_tree2':'DOST tree'}
feats = {}
for vesname in eval_vesname:
    feats[vesname] = {}
    
for pi in pilist[:]:
    icafem = iCafe(icafe_dir+'/'+pi)
    if not icafem.existPath('metric.pickle'):
        continue
        
    with open(icafem.getPath('metric.pickle'),'rb') as fp:
        all_metric_dict = pickle.load(fp)
    
    #for vesname in all_metric_dict:
    for vesname in eval_vesname:
        if vesname not in all_metric_dict:
            print('no',vesname,'in',pi)
            continue
        for metric in all_metric_dict[vesname]:
            if metric not in feats[vesname]:
                feats[vesname][metric] = []
            feats[vesname][metric].append(all_metric_dict[vesname][metric]) 
    

sel_metrics = ['OV','AI', 'MOTA', 'IDF1', 'IDS']
print('\t'.join(['']+sel_metrics))
for vesname in feats:
    featstr = eval_vesname[vesname]+'\t'
    for metric in sel_metrics:
        if metric in ['IDS']:
            featstr += '%.1f\t'%np.mean(feats[vesname][metric])            
        else:
            featstr += '%.3f\t'%np.mean(feats[vesname][metric])
    print(featstr)

In [ ]:
#harborbiew
#collect feats from pickle
#'frangi_ves':'Frangi','seg_unet':'U-Net','seg_raw':'DDT',
                #'raw_sep':'iCafe','cnn_snake':'CNN Tracker','dcat_snake':'DCAT',
eval_vesname = {'tree_main':'DOST (ours)'}
feats = {}
for vesname in eval_vesname:
    feats[vesname] = {}
    
for pi in pilist[:]:
    icafem = iCafe(icafe_dir+'/'+pi)
    if not icafem.existPath('metric.pickle'):
        continue
        
    with open(icafem.getPath('metric.pickle'),'rb') as fp:
        all_metric_dict = pickle.load(fp)
    
    #for vesname in all_metric_dict:
    for vesname in eval_vesname:
        if vesname not in all_metric_dict:
            print('no',vesname,'in',pi)
            continue
        for metric in all_metric_dict[vesname]:
            if metric not in feats[vesname]:
                feats[vesname][metric] = []
            feats[vesname][metric].append(all_metric_dict[vesname][metric]) 
    

sel_metrics = ['OV','AI', 'MOTA', 'IDF1', 'IDS']
print('\t'.join(['']+sel_metrics))
for vesname in feats:
    featstr = eval_vesname[vesname]+'\t'
    for metric in sel_metrics:
        if metric in ['IDS']:
            featstr += '%.1f\t'%np.mean(feats[vesname][metric])            
        else:
            featstr += '%.3f\t'%np.mean(feats[vesname][metric])
    print(featstr)

In [ ]:
feats[vesname].keys()